In [13]:
import librosa
import matplotlib.pyplot as plt
import IPython.display as dis
import numpy as np
import os, sys
from tqdm import tqdm
import pandas as pd
import librosa.display

In [3]:
rawdata = pd.read_csv('Raw_data.csv')

In [6]:
rawdata

,song_id,song_name,song_lyric,senti_tag
0,32399836,Respect,Put your hands in the air\n(Put your hands in ...,화나요
1,31052006,Drink Slow Henny,Underground studio\n쿠바나 더블 불 붙이고\n오늘 밤은 가자 로데오...,화나요
2,4196651,Badman,어둠 속에서 들리는 절규 \n공포에 질린 세상을 \n뒤집어버릴거야 다 \n미쳐버린 ...,화나요
3,31448485,위로 해줄래,"이렇게 달이 아름다운 밤이 오면\n감정이 올라 So, lonely\n며칠간 묵었던 ...",화나요
4,30773556,Hate Me,I know you hate me\nI know you hate me\nI know...,화나요
...,...,...,...,...
1332,31835675,Help Me,No lyric,화나요
1333,33193810,MOON,달이 밝은 밤 숨지 못한 나\n붉은색의 달빛에 내 눈물이\n내 표정이 보여\nDo ...,화나요
1334,30287019,롤린 (Rollin'),그 날을 잊지 못해 babe\n날 보며 환히 웃던 \n너의 미소에\n홀린 듯 \nI...,화나요
1335,34943312,나의 X에게,우리 다시 만날래\n예쁘게 빛나던\n모든 추억들이\n너무 그리워\n너의 품에 안길래...,화나요


In [18]:
# 경로 설정
path = 'C:\\Users\\인공지능사관학교\\Desktop\\최종프로젝트\\data_wav'

# melspectrogram 을 numpy 배열 형태로 저장
for i in tqdm(range(len(rawdata))):
    music_path = path + '\\' + str(rawdata['song_id'].iloc[i]) + '.wav'
    y, sr = librosa.load(music_path)
    S = librosa.feature.melspectrogram(y=y, sr=sr)
    S_DB = librosa.amplitude_to_db(S, ref=np.max)
    
    np.save('C:\\Users\\인공지능사관학교\\Desktop\\최종프로젝트\\melspectrogram_data\\'+str(rawdata['song_id'].iloc[i])+'.npy',
            S_DB)
    



100%|██████████| 1337/1337 [2:09:06<00:00,  5.79s/it] 


In [25]:
lst = os.listdir('../melspectrogram_data')
data_x = []
data_y = []
for i in tqdm(range(len(rawdata))):
   data_x.append(np.load('../melspectrogram_data/' + str(rawdata['song_id'].iloc[i])+'.npy'))
   data_y.append(rawdata['senti_tag'].iloc[i])

100%|██████████| 1337/1337 [00:14<00:00, 93.69it/s]


In [26]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data_y = label_encoder.fit_transform(data_y)

In [43]:
wav_to_img_data = []
target = []

# 128*128 사이즈로 멜스펙토그램 이미지를 잘라서 담아두고
# 타겟값을 같이 추가한 넘파이배열을 만든다
for idx in tqdm(range(len(data_x))):
    for i in range(15):
        wav_to_img_data.append(data_x[idx][:, 128* i : 128 * (i+1)].reshape(128,128,-1))
        target.append(data_y[idx])
wav_to_img_data = np.array(wav_to_img_data)
target = np.array(target)

100%|██████████| 1337/1337 [00:00<00:00, 67031.45it/s]


In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, BatchNormalization, Activation, Flatten

In [47]:
# layers

inputs = Input(shape=(128,128,1))

conv_1 = Conv2D(16, kernel_size=3, strides=1, padding='same')(inputs)
bn_1 = BatchNormalization()(conv_1)
relu_1 = Activation('relu')(bn_1)
maxpool_1 = MaxPooling2D(pool_size=2, padding='same')(relu_1)

conv_2 = Conv2D(32, kernel_size=3, strides=1, padding='same')(maxpool_1)
bn_2 = BatchNormalization()(conv_2)
relu_2 = Activation('relu')(bn_2)
maxpool_2 = MaxPooling2D(pool_size=2, padding='same')(relu_2)

conv_3 = Conv2D(64, kernel_size=3, strides=1, padding='same')(maxpool_2)
bn_3 = BatchNormalization()(conv_3)
relu_3 = Activation('relu')(bn_3)
maxpool_3 = MaxPooling2D(pool_size=2, padding='same')(relu_3)

flatten = Flatten()(maxpool_3)
dense_1 = Dense(128, activation='relu')(flatten)
dense_2 = Dense(6, activation='softmax')(dense_1)

In [48]:
model = tf.keras.models.Model(inputs, dense_2)

In [49]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [52]:
history = model.fit(wav_to_img_data, target, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
502/502 [==============================] - 235s 467ms/step - loss: 1.0427 - acc: 0.5911 - val_loss: 2.1568 - val_acc: 0.1488
Epoch 2/30
502/502 [==============================] - 252s 502ms/step - loss: 0.9641 - acc: 0.6239 - val_loss: 2.9184 - val_acc: 0.1416
Epoch 3/30
502/502 [==============================] - 232s 463ms/step - loss: 0.8740 - acc: 0.6634 - val_loss: 7.6099 - val_acc: 0.2144
Epoch 4/30
502/502 [==============================] - 231s 461ms/step - loss: 0.8013 - acc: 0.6906 - val_loss: 2.5520 - val_acc: 0.2408
Epoch 5/30
502/502 [==============================] - 229s 456ms/step - loss: 0.7302 - acc: 0.7222 - val_loss: 6.1251 - val_acc: 0.1755
Epoch 6/30
502/502 [==============================] - 228s 454ms/step - loss: 0.6611 - acc: 0.7539 - val_loss: 5.0649 - val_acc: 0.1212
Epoch 7/30
502/502 [==============================] - 228s 455ms/step - loss: 0.6195 - acc: 0.7671 - val_loss: 2.1510 - val_acc: 0.2498
Epoch 8/30
502/502 [============================

In [63]:
model.save('3depth_CNN_model.h5')